# Goal
We want only the latest status for every issued loan to predict if it is a risk or not. We want to create a new dataset that has only the important columns and the latest records.

This will drastically reduce our dataset and hopefully we will be able to handle it better.

In [1]:
# loading important libraries

import findspark
findspark.init()
import pyspark as ps
import warnings
from pyspark.sql import SQLContext

In [ ]:
try:
    sc = ps.SparkContext('local[*]')
    sqlContext = SQLContext(sc)
    print("Just created a SparkContext")
except ValueError:
    warnings.warn("SparkContext already exists in this scope")

In [2]:
perfsub=spark.read.csv('cmpnydata/perfSubset.csv',header='true',inferSchema='true')

In [ ]:
perfsub.createOrReplaceTempView('psub')

In [ ]:
x=spark.sql("SELECT loanID,currIntRate,zeroBalCode,zeroBalEffDate,lastPaidInstallmentDate,currActualUPB,loanAge,remainingMonthsToLegalMaturity,adjustedMonthsToMaturity,maturityDate,msa,currLoanDelinquencyStatus,modFlag,lastPaidInstallmentDate,foreclosureDate,dispositionDate,foreclosureCosts,otherForeclosureProceeds,nonInterestBearingUPB,foreclosurePrincipalWriteOffAmt,ROW_NUMBER() OVER(PARTITION BY loanID ORDER BY remainingMonthsToLegalMaturity) AS ranking  FROM psub GROUP BY loanID,currIntRate,zeroBalCode,zeroBalEffDate,lastPaidInstallmentDate,currActualUPB,loanAge,remainingMonthsToLegalMaturity,adjustedMonthsToMaturity,maturityDate,msa,currLoanDelinquencyStatus,modFlag,lastPaidInstallmentDate,foreclosureDate,dispositionDate,foreclosureCosts,otherForeclosureProceeds,nonInterestBearingUPB,foreclosurePrincipalWriteOffAmt")

x.createOrReplaceTempView('df')

df1=sqlContext.sql("SELECT loanID,currIntRate,zeroBalCode,zeroBalEffDate,lastPaidInstallmentDate,currActualUPB,loanAge,remainingMonthsToLegalMaturity,adjustedMonthsToMaturity,maturityDate,msa,currLoanDelinquencyStatus,modFlag,foreclosureDate,dispositionDate,foreclosureCosts,otherForeclosureProceeds,nonInterestBearingUPB,foreclosurePrincipalWriteOffAmt FROM df where ranking=1")

df1.coalesce(1).write.save('/mnt/d/Hack_UTD/cmpnydata/loanperf1.csv',format='csv')

# End!

We have successfully extracted the **LATEST records for every LoanID** since that is the only record we're interested in and we have also taken a subset of available columns based on our study of Glossary.